In [9]:
import numpy as np
import pandas as pd

from params import Params
from gpkf import Gpkf
from generateSyntheticData import generateSyntheticData

In [7]:
def start_train():
    params = Params()
    model = Gpkf(params)

    #F, Y, noiseVar = generateSyntheticData(params.data)
    
    F, Y, noiseVar = format_data()

    # GPKF estimate
    posteriorMean, posteriorCov, logMarginal = model.estimation(Y, noiseVar)

    #GPKF prediction
    predictedMean, predictedCov = model.prediction(Y, noiseVar)

    return model

In [10]:
model = start_train()

1 dim
1 dim
